In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
from tensorflow import keras
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore

In [6]:
import numpy as np
data = np.load('oxflower17.npz')

In [7]:
data

NpzFile 'oxflower17.npz' with keys: X, Y

In [8]:
X = data['X']

In [9]:
y = data['Y']

In [10]:
X

array([[[[0.19607843, 0.25882354, 0.15686275],
         [0.18039216, 0.24313726, 0.13725491],
         [0.15294118, 0.21568628, 0.10980392],
         ...,
         [0.12941177, 0.12156863, 0.12156863],
         [0.13725491, 0.12941177, 0.14117648],
         [0.14509805, 0.13725491, 0.14901961]],

        [[0.1882353 , 0.2509804 , 0.15294118],
         [0.1764706 , 0.23529412, 0.13333334],
         [0.15686275, 0.21176471, 0.11372549],
         ...,
         [0.11372549, 0.10588235, 0.10588235],
         [0.12941177, 0.12156863, 0.13333334],
         [0.14117648, 0.13333334, 0.14509805]],

        [[0.17254902, 0.22352941, 0.14901961],
         [0.16078432, 0.21176471, 0.13333334],
         [0.14901961, 0.19607843, 0.11372549],
         ...,
         [0.08627451, 0.07843138, 0.07843138],
         [0.10588235, 0.09803922, 0.10980392],
         [0.12156863, 0.11372549, 0.1254902 ]],

        ...,

        [[0.20784314, 0.2509804 , 0.32941177],
         [0.20784314, 0.25490198, 0.3254902 ]

In [11]:
y

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [12]:
x_train = X.astype('float32')
y_train = y

In [10]:
x_train

array([[[[0.19607843, 0.25882354, 0.15686275],
         [0.18039216, 0.24313726, 0.13725491],
         [0.15294118, 0.21568628, 0.10980392],
         ...,
         [0.12941177, 0.12156863, 0.12156863],
         [0.13725491, 0.12941177, 0.14117648],
         [0.14509805, 0.13725491, 0.14901961]],

        [[0.1882353 , 0.2509804 , 0.15294118],
         [0.1764706 , 0.23529412, 0.13333334],
         [0.15686275, 0.21176471, 0.11372549],
         ...,
         [0.11372549, 0.10588235, 0.10588235],
         [0.12941177, 0.12156863, 0.13333334],
         [0.14117648, 0.13333334, 0.14509805]],

        [[0.17254902, 0.22352941, 0.14901961],
         [0.16078432, 0.21176471, 0.13333334],
         [0.14901961, 0.19607843, 0.11372549],
         ...,
         [0.08627451, 0.07843138, 0.07843138],
         [0.10588235, 0.09803922, 0.10980392],
         [0.12156863, 0.11372549, 0.1254902 ]],

        ...,

        [[0.20784314, 0.2509804 , 0.32941177],
         [0.20784314, 0.25490198, 0.3254902 ]

In [11]:
y_train

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [12]:
x_train.shape, y_train.shape

((1360, 224, 224, 3), (1360, 17))

In [24]:
import tensorflow as tf
x_train = tf.image.resize(x_train, (244, 244))

In [25]:
# Residual Block
def residual_block(x, filters, downsample = False):
    strides = (2,2) if downsample else (1,1)
    
    # First Concolution layer of the block
    y = Conv2D(filters, kernel_size = (3,3), strides = strides, padding = 'same')(x)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    
    # Second convolution layer of the block
    y = Conv2D(filters, kernel_size = (3,3), strides = (1,1), padding = 'same')(y)
    y = BatchNormalization()(y)
    
    # Skip connection if downsample or number of filters change
    if downsample:
        x = Conv2D(filters, kernel_size = (1,1), strides = (2,2), padding = 'same')(x)
        
    # Add skip connection 
    y = Add()([x,y])
    y = Activation('relu')(y)
    return y


# Build the ResNet model
def resnet(input_shape, num_classes):
    inputs = keras.Input(shape = input_shape)
    
    # Initial convolution layer
    x = Conv2D(16, kernel_size = (3,3), strides = (1,1), padding = 'same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Residual blocks
    x = residual_block(x, filters = 16)
    x = residual_block(x, filters = 16)
    x = residual_block(x, filters = 32, downsample=True)
    x = residual_block(x, filters = 32)
    x = residual_block(x, filters = 64, downsample=True)
    x = residual_block(x, filters = 64)
    
    # Average Pooling and output layer
    x = AveragePooling2D(pool_size = (8,8))(x)
    x = Flatten()(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs = inputs, outputs = outputs)
    return model

In [ ]:
model = resnet(input_shape = (244,244,3), num_classes = 17)

model.compile(optimizer = Adam(learning_rate=0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(x_train, y_train, batch_size = 64, epochs = 5, verbose = 1, validation_split = 0.2, shuffle = True)

In [14]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.models import Sequential, Model

base_model = ResNet50(weights = 'imagenet', include_top = False, input_shape = (224,224,3))

for layer in base_model.layers:
    layer.trainable = False
    
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(17, activation='softmax')(x)

model = Model(inputs = base_model.inputs, outputs = outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(x_train, y_train, batch_size = 64, validation_split = 0.2, shuffle = True)

c:\Users\91808\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_179']
Received: inputs=Tensor(shape=(64, 224, 224, 3))
  warnings.warn(msg)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.0600 - loss: 0.4877

c:\Users\91808\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_179']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


17/17 ━━━━━━━━━━━━━━━━━━━━ 74s 4s/step - accuracy: 0.0599 - loss: 0.4803 - val_accuracy: 0.0588 - val_loss: 0.2539


In [16]:
model

<Functional name=functional_1, built=True>